# Agenda (Day4 : 09Oct2025):    
    
    - More towards Parallel-programming(hardware & packages)
    - Models and Frameworks of Python Parallel-Programming
    - cupy intro, and openMPI intro
    - lab installation issues (Siddhant)
    


# Parallel Computing Memory-architecture

In [6]:

#SISD
print("#SISD ")
for i in range(5):
    print(f'SISD processing item {i}')

print("+- " * 25)
print("#SIMD ")
# SIMD
import numpy as np
arr = np.array([1,2,3,4,5])
print(" SIMD: ", arr * 2)

print("+- " * 25)
print("#MISD ")
# MISD
import threading

def add(data_structure1, data_structure2) : print(f'Contatenate: { data_structure1 + data_structure2}') 
def sq(x) : print(f'square: { x * x}')
def cube(x) : print(f'cube: { x **3}')

data_tup = (3, )
t1 = threading.Thread(target = sq, args=data_tup)
t2 = threading.Thread(target = cube, args=data_tup)

t1.start(); t2.start();

print("+- " * 25)
print("#MIMD ")
data_tup = ([1,2,3,4], [5,6,7,8])
t3 = threading.Thread(target = add, args=data_tup)

t3. start()

t1.join(); t2.join(); t3.join()

#SISD 
SISD processing item 0
SISD processing item 1
SISD processing item 2
SISD processing item 3
SISD processing item 4
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
#SIMD 
 SIMD:  [ 2  4  6  8 10]
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
#MISD 
square: 9
cube: 27
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
#MIMD 
Contatenate: [1, 2, 3, 4, 5, 6, 7, 8]


# Distributed Memory

In [ ]:
import mpi4py 

In [7]:

def add(x, y):
    print(' -- add called --')
    return x + y

def repeater(fn, src): # Functional-style-programming
    first = src[0]

    for second in src[1:]:
        first = fn(first, second)
    return first
    
def main():
    lst1 = [ val for val in range(10) ] 
    print('lst1 = ' , lst1)

    result = repeater(add, lst1)
    print('result = ', result)


    
if __name__ == '__main__':
    main()



lst1 =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
result =  45


In [8]:

def add(x, y):
    print(' -- add called --')
    return x + y

def repeater(fn, src): # Functional-style-programming
    first = src[0]

    for second in src[1:]:
        first = fn(first, second)
    return first
    
def main():
    lst1 = [ val for val in range(10) ] 
    print('lst1 = ' , lst1)

    result = sum( lst1)
    print('result = ', result)


    
if __name__ == '__main__':
    main()



lst1 =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
result =  45


In [9]:

def add(x, y):
    print(' -- add called --')
    return x + y

def main():
    import functools
    lst1 = [ val for val in range(10) ] 
    print('lst1 = ' , lst1)

    result = functools.reduce( add, lst1)
    print('result = ', result)


    
if __name__ == '__main__':
    main()



lst1 =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
 -- add called --
result =  45


In [10]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def mpirun(line, cell):
    args = line.split()
    with open("temp_mpi.py", "w") as f:
        f.write(cell)
    cmd = ["mpiexec" ] + args + ["python", "temp_mpi.py"]
    subprocess.run(cmd)
    



In [11]:
%%mpirun -n 4

def main():
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    print(f' Process {rank} out of {size} is alive")
    
if __name__ == '__main__':
    main()

NameError: name 'subprocess' is not defined

## Asyncronous programming

In [12]:
# Task based parallism

import asyncio
import time

async def worker(name):
    print(f"{name} started")
    await asyncio.sleep(2) #time.sleep(2)
    print(f"{name} end")

async def main():
    t1 = asyncio.create_task(worker("Task1111"))
    t2 = asyncio.create_task(worker("Task2222"))

    await t1
    await t2

if '__main__' == __name__:
    asyncio.run(main())
    

RuntimeError: asyncio.run() cannot be called from a running event loop

## Thread in a sub-class

In [13]:
import threading, time

class WorkerThread( threading.Thread):
    def __init__(self, n, **kargs):
        super().__init__( **kargs)
        self.m_n = n

    def run(self):
        for i in range(self.m_n):
            print(f"{self.name} => { i}")
            time.sleep(0.2)

if '__main__' == __name__:
    t = WorkerThread(6, name="MyWorkerThreadFunction")
    t.start()
    t.join()

MyWorkerThreadFunction => 0
MyWorkerThreadFunction => 1
MyWorkerThreadFunction => 2
MyWorkerThreadFunction => 3
MyWorkerThreadFunction => 4
MyWorkerThreadFunction => 5


In [16]:



print("Concept: RLock Threads.")
print("+- "*25)


import threading, time

# Note : Always bind Resource and lock together, unlike below global
count = 0
lock = threading.Lock()

def increment(): # with Lock
    global count
    for _ in range(1000000):
        with lock:
            count += 1
        # lock.unlock()

def with_RLock(rlock):
    with rlock:
        print(' rlock acquired here!' )
        with rlock:
            print('Re-acquired by same thread')

if '__main__' == __name__:
    rlock = threading.RLock()

    with_RLock(rlock)

    print(" End")

Concept: RLock Threads.
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
 rlock acquired here!
Re-acquired by same thread
 End


In [17]:
help(threading.Semaphore)

Help on class Semaphore in module threading:

class Semaphore(builtins.object)
 |  Semaphore(value=1)
 |
 |  This class implements semaphore objects.
 |
 |  Semaphores manage a counter representing the number of release() calls minus
 |  the number of acquire() calls, plus an initial value. The acquire() method
 |  blocks if necessary until it can return without making the counter
 |  negative. If not given, value defaults to 1.
 |
 |  Methods defined here:
 |
 |  __enter__ = acquire(self, blocking=True, timeout=None)
 |
 |  __exit__(self, t, v, tb)
 |
 |  __init__(self, value=1)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  acquire(self, blocking=True, timeout=None)
 |      Acquire a semaphore, decrementing the internal counter by one.
 |
 |      When invoked without arguments: if the internal counter is larger than
 |      zero on entry, decrement it by one and return immediately. If it is zero
 |      o

## Thread Synchronization using conditional-variables

In [20]:


print("Concept: Thread Synchronization using conditional-variables.")
print("+- "*25)

import threading, queue

cond_var = threading.Condition()
items = [] 

def producer():
    for i in range(4):
        with cond_var:
            items.append(i)
            print("Produced:", i)
            
            cond_var.notify() # notifies single-thread in wait-state.
            #cond_var.notifyall() # notifies all-threads in wait-state.
            time.sleep(1)
def consumer():
    while True:
        with cond_var:
            while not items:
                cond.wait()
            item = items.pop(0)
            
            print("Consumed:", item)
            if item == 3: break
def main():
    t1 = threading.Thread(target=producer )
    t2 = threading.Thread(target=consumer )
    t1.start(); t2.start()
    t1.join() 
    t2.join()

if __name__ == '__main__':
    main()


Concept: Thread Synchronization using conditional-variables.
+- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- +- 
Produced: 0
Produced: 1
Produced: 2
Produced: 3
Consumed: 0
Consumed: 1
Consumed: 2
Consumed: 3


### multiprocessing.Pipe

In [ ]:
from multiprocessing import Process, Pipe

def sender(conn):
    pass

def receiver(conn):
    pass

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()

    p1 = Process(target = sender, args = (parent_conn, ))
    p2 = Process(target = receiver, args = (child_conn, ))

    p1.start(); p2.start(); p1.join(); p2.join()

In [26]:
import requests
import time
import concurrent.futures

img_urls = [
    'https://images.unsplash.com/photo-1524429656589-6633a470097c',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1524429656589-6633a470097c',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1524429656589-6633a470097c',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1549692520-acc6669e2f0c'
]


def download_images( img_uri):
    img_bytes = requests.get(img_uri).content
    img_name = img_uri.split('/')[3]
    img_name = img_name + ".jpg"

    with open(img_name, "wb") as img_file:
        img_file.write(img_bytes)
        

def main():
    #t1 = time.perf_counter() # Use ThreadPoolExecutor()
    t1 = time.time()
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=3)
    result = executor.map(download_images, img_urls)
    t2 = time.time() #perf_counter()

    print(f' Threading time consumption: Finished in {t2-t1} seconds')


    # Single
    t1 = time.time() #perf_counter()
    for img_uri in img_urls:
        download_images(img_uri)
        
    t2 = time.time() #perf_counter()
    print(f' Serial time consumption: Finished in {t2-t1} seconds')

    

if __name__ == '__main__':
    main()

 Threading time consumption: Finished in 0.004340648651123047 seconds
 Serial time consumption: Finished in 7.689591646194458 seconds
